Этот ноутбук для выделения из общей базы нескольких предприятий для последующей проверки работы модели. Нужно взять такие предприятия, чтобы у одних был статус банкрота, а других нет.

In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
# загрузим данные об активах и задолженности компаний
df_accounts = pd.read_csv('df_accounts_sample.csv', sep=',')

# приведем в порядок имена колонок и типы данных, значения NaN заменим нулями
df_accounts.rename(columns={'Unnamed: 0': 'id'}, inplace=True)
df_accounts.set_index('id', inplace=True)
df_accounts.fillna(0, inplace=True)
df_accounts = df_accounts.astype({'year': 'int16',
                                  'short_term_liabilities_fiscal_year': 'float64',
                                  'balance_assets_fiscal_year': 'float64',
                                  'okei': 'int16'})
df_accounts

,inn,year,long_term_liabilities_fiscal_year,short_term_liabilities_fiscal_year,balance_assets_fiscal_year,okei
id,,,,,,
0,2465002653,2012,0.0,113756.0,122240.0,384
1,3505003270,2012,21461.0,20211.0,81132.0,384
2,3517003250,2012,0.0,0.0,48063.0,384
3,5501078737,2012,0.0,288763.0,419522.0,384
4,6140019517,2012,0.0,0.0,11089.0,384
...,...,...,...,...,...,...
1030,7842353970,2020,0.0,2548.0,16411.0,384
1031,8908001998,2019,0.0,8636.0,130731.0,384
1032,8908001998,2020,0.0,14318.0,128891.0,384


In [3]:
# загрузим данные о компаниях, претерпевших банкротство
df_bankruptcies = pd.read_csv('df_bankruptcies_sample.csv', sep=',')

# приведем в порядок имена колонок и типы данных
df_bankruptcies.rename(columns={'Unnamed: 0': 'id'}, inplace=True)
df_bankruptcies.set_index('id', inplace=True)
df_bankruptcies = df_bankruptcies.astype({'bancrupt_year': 'int16'})
df_bankruptcies

,bankrupt_id,bancrupt_year,inn
id,,,
0,79546,2015,5620006322
1,76594,2015,2111008200
2,94301,2016,5414100369
3,93799,2019,2347009476
4,199671,2018,7224008489
...,...,...,...
101,530816,2021,7728812071
102,325608,2019,6154086295
103,339365,2020,6627020656


In [4]:
# соберем все сведения о предприятиях в единый датафрейм 'df_firms'
df_firms = pd.merge(df_accounts, df_bankruptcies, left_on=['inn'], right_on=['inn'], how='outer')
df_firms.fillna(0, inplace=True)

# приведем в порядок типы данных
df_firms = df_firms.astype({'year': 'int16',
                            'long_term_liabilities_fiscal_year': 'float64',
                            'short_term_liabilities_fiscal_year': 'float64',
                            'balance_assets_fiscal_year': 'float64',
                            'bancrupt_year': 'int16'})
df_firms

,inn,year,long_term_liabilities_fiscal_year,short_term_liabilities_fiscal_year,balance_assets_fiscal_year,okei,bankrupt_id,bancrupt_year
0,2465002653,2012,0.0,113756.0,122240.0,384,527279.0,2021
1,2465002653,2013,0.0,212584.0,241577.0,384,527279.0,2021
2,2465002653,2017,0.0,51779.0,76710.0,384,527279.0,2021
3,2465002653,2014,0.0,63670.0,95167.0,384,527279.0,2021
4,2465002653,2015,0.0,25896.0,52292.0,384,527279.0,2021
...,...,...,...,...,...,...,...,...
1030,2721208520,2019,50.0,6242.0,80027.0,384,444691.0,2020
1031,2721208520,2020,50.0,21989.0,94781.0,384,444691.0,2020
1032,6311175195,2018,0.0,2086.0,4264.0,384,0.0,0
1033,5042125258,2019,12.0,144054.0,24298.0,384,259307.0,2019


In [5]:
df_firms_agg = df_firms.sort_values(by=['inn','year']).groupby('inn', as_index=False).\
    agg(year=('year', 'last'), bancrupt_year=('bancrupt_year', 'last'))

In [6]:
df_firms_agg['bankrupt'] = [1 if x != 0 else 0 for x in df_firms_agg['bancrupt_year']]
df_firms_agg.drop(['year', 'bancrupt_year'], axis=1, inplace=True)
df_firms_agg.set_index('inn', inplace=True)

In [7]:
df_firms_agg

,bankrupt
inn,
1001003743,0
1020010136,1
1203000142,1
1302000640,1
1411004211,1
...,...
8617004112,1
8901998318,0
8908001998,1


Возьмем для примера два предприятия: 1302000640, 9102060446

In [8]:
test_firms_list = [1302000640, 9102060446]

df_firms_agg[df_firms_agg.index.isin(test_firms_list)]

,bankrupt
inn,
1302000640,1
9102060446,0


In [9]:
# запишем информацию о выбранных предприятиях в отдельные csv-файлы
if not os.path.isdir('test'):
     os.mkdir('test')
for firm_inn in test_firms_list:
    df_accounts_select = df_accounts[(df_accounts['inn'] == firm_inn)].reset_index().drop('id', axis=1)
    df_accounts_select.to_csv(f'test/{firm_inn}.csv')#, index=False)

In [10]:
# удалим информацию из общего файла о выбранных предприятиях 
# и запишем ее в новый файл 'df_accounts_sample_modify.csv'
df_accounts_modify = df_accounts[~(df_accounts['inn'].isin(test_firms_list))].reset_index().drop('id', axis=1)
df_accounts_modify.to_csv('df_accounts_sample_modify.csv')#, index=False)

### Примечание:
необходимо убрать из папки 'success' соответствующие выбранным предприятиям pkl-файлы.